In [1]:
pip install mip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 32.9 MB/s eta 0:00:00


In [2]:
import numpy as np
import urllib.request 
from mip import Model, xsum, maximize, BINARY, LinExpr

In [3]:
def CTFPFunction(graphUrl, skillsUrl, secondarySkillsUrl, numTeams, teamUsersProportion, graphSep = "\n", numUsersSep = " ", skillsSep = "\n", numSkillsSep = " "):

  #1 - CRIANDO MATRIZ DE ADJACÊNCIA DAS RELAÇÕES ENTRE OS USUÁRIOS
  graph = urllib.request.urlopen(graphUrl).read().decode('utf-8').split(graphSep)
  del graph[-1] #vazio

  numUsers = int(graph[0].split(numUsersSep)[0])
  relationships = np.zeros(numUsers*numUsers, dtype=np.int64).reshape(numUsers,numUsers)

  for line in graph[1:]:
    l = [x for x in line.split(" ") if x]
    l[0] = int(l[0])
    l[1] = int(l[1])
    l[2] = round(float(l[2]))
    if int(l[0]) < numUsers and int(l[1]) < numUsers:
      relationships[l[0],l[1]] = relationships[l[1],l[0]] = int(l[2])

  print(f'Número usuários: {numUsers}')
  print(f'Matriz de relacionamentos {relationships}')
  print('\n')


  #2 - DEFININDO CONJUNTO DE SKILLS DE CADA USUÁRIO
  skillsData = urllib.request.urlopen(skillsUrl).read().decode('utf-8').split(skillsSep)
  numSkills = int(skillsData[0].split(numSkillsSep)[0])

  print(f'Número de skills: {numSkills}')
  lstSkills = skillsData[0].split(numSkillsSep)[1:]
  mapSkills = {int(lstSkills[i].split(":")[0]):lstSkills[i].split(":")[1] for i in range(len(lstSkills))}
  print(f'Lista de skills: {mapSkills}')
  print('\n')


  #3 - CONTANDO O NÚMERO DE USUÁRIOS PARA CADA SKILL
  usersSkillsMatrix = np.zeros(numUsers*numSkills, dtype=np.int64).reshape(numUsers,numSkills)

  for index,item in enumerate(skillsData[1:]):
    usersSkillsMatrix[index] = np.asarray(item.split(" "))

  numUsersBySkill = usersSkillsMatrix.sum(axis=0)


  #4 - FUNÇÃO QUE RETORNA O CONJUNTO DE SKILLS DE UM USUÁRIO
  def userSkillSet(user):
    return np.asarray([id for id,s in enumerate(skillsData[user+1].split(" ")) if s and int(s) == 1])

  
  #5 - DEFININDO DEMANDA DE USUÁRIOS POR SKILL PARA CADA TIME
  teamDemand = np.full((numTeams, numSkills), 1, dtype=np.int64)

  for index,item in enumerate(numUsersBySkill):
    for num in range(numTeams):
      teamDemand[num][index] = round(item/teamUsersProportion)

  print(f'Demanda de cada time: {teamDemand}')
  print('\n')

  #6 - CONJUNTO DE SKILLS QUE CADA TIME PRECISA
  def demandSkillSet(team):
    return np.asarray([id for id,s in enumerate(teamDemand[team]) if s > 0])


  #7 - IMPLEMENTAÇÃO DO MODELO
  model = Model("CTFP")

  #7.1 - Variáveis
  x = [[[model.add_var(var_type=BINARY) for s in range(numSkills)] for u in range(numUsers)] for t in range(numTeams)]

  #7.2 - Função objetivo
  model.objective = maximize(xsum( xsum( xsum( x[t][u][s] for s in userSkillSet(u) ) for u in range(numUsers) ) for t in range(numTeams)))

  #7.3 - Restrições

  #restrição (63)
  for t in range(numTeams):
    for u in range(numUsers):
      for v in range(numUsers):
        if relationships[u,v] == -1:
          model += xsum(x[t][u][s] for s in userSkillSet(u)) + xsum(x[t][v][s] for s in userSkillSet(v)) <= 1

  #restrição (64)
  for t1 in range(numTeams):
    for t2 in np.arange(t1+1, numTeams, 1):
      for u in range(numUsers):
        for v in range(numUsers):
          if relationships[u,v] == 1:
            model += xsum(x[t1][u][s] for s in userSkillSet(u)) + xsum(x[t2][v][s] for s in userSkillSet(v)) <= 1

  #restrição (65)
  for u in range(numUsers):
    model += xsum( xsum( x[t][u][s] for s in userSkillSet(u)) for t in range(numTeams)) <= 1

  #restrição (66)
  for t in range(numTeams):
    for s in demandSkillSet(t):
      model += xsum( x[t][u][s] for u in range(numUsers) if s in userSkillSet(u)) >= teamDemand[t,s]

  model.optimize()

  print('\n')

  result = model.objective_value

  print(f'Total de indivíduos alocados: {result}\n')

  if result == None:
    return

  secondarySkillsData = urllib.request.urlopen(secondarySkillsUrl).read().decode('utf-8').split(skillsSep)
  numSecondarySkills = int(secondarySkillsData[0].split(numSkillsSep)[0])

  lstSecondarySkills = secondarySkillsData[0].split(numSkillsSep)[1:]
  mapSecondarySkills = {int(lstSecondarySkills[i].split(":")[0]):lstSecondarySkills[i].split(":")[1] for i in range(len(lstSecondarySkills))}

  def userSecondarySkillSet(user):
    return np.asarray([id for id,s in enumerate(secondarySkillsData[user+1].split(" ")) if s and int(s) == 1])

  for teamIdx,team in enumerate(x):
    usersCount = 0
    skillsUsersCount = [0] * numSkills
    secondarySkillsUserCount = [0] * numSecondarySkills
    for userIdx,user in enumerate(team):
      for skillIdx,skill in enumerate(user):
        skillsUsersCount[skillIdx] += skill.x
        usersCount += skill.x
        for s in userSecondarySkillSet(userIdx):
          secondarySkillsUserCount[s] += skill.x
    
    print(f'Resultados da equipe {teamIdx}')

    skillsUsersCountStr = ' | '.join(mapSkills[ind]+': '+str(count) for ind, count in enumerate(skillsUsersCount))
    secondarySkillsUsersCountStr = ' | '.join(mapSecondarySkills[ind]+': '+str(count) for ind, count in enumerate(secondarySkillsUserCount))
    print(f'Número de indivíduos na equipe: {usersCount}')
    print(f'Número de indíviduos em cada skill: {skillsUsersCountStr}')
    print(f'Número de indíviduos em cada skill secundária: {secondarySkillsUsersCountStr}')

    print('\n')

## 2010

In [4]:
#duas equipes

CTFPFunction('https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/congress-2010-v1.g',
             'https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/state-as-skill/2010.txt',
             'https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/party-as-skill/2010.txt',
             2,
             5,
             numUsersSep="\t")

Número usuários: 545
Matriz de relacionamentos [[0 1 1 ... 0 1 1]
 [1 0 1 ... 0 1 0]
 [1 1 0 ... 0 1 1]
 ...
 [0 0 0 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 0 1 ... 0 0 0]]


Número de skills: 27
Lista de skills: {0: 'MG', 1: 'SP', 2: 'RJ', 3: 'BA', 4: 'PE', 5: 'RR', 6: 'MS', 7: 'PR', 8: 'SC', 9: 'RS', 10: 'MT', 11: 'ES', 12: 'DF', 13: 'CE', 14: 'TO', 15: 'GO', 16: 'AP', 17: 'PA', 18: 'AM', 19: 'RO', 20: 'AC', 21: 'AL', 22: 'MA', 23: 'RN', 24: 'PB', 25: 'SE', 26: 'PI'}


Demanda de cada time: [[11 14 10  9  5  2  2  6  4  6  2  2  2  5  2  3  2  4  2  2  2  2  4  2
   2  2  2]
 [11 14 10  9  5  2  2  6  4  6  2  2  2  5  2  3  2  4  2  2  2  2  4  2
   2  2  2]]




Total de indivíduos alocados: 331.0

Resultados da equipe 0
Número de indivíduos na equipe: 157.0
Número de indíviduos em cada skill: MG: 14.0 | SP: 23.0 | RJ: 19.0 | BA: 11.0 | PE: 7.0 | RR: 2.0 | MS: 4.0 | PR: 10.0 | SC: 4.0 | RS: 8.0 | MT: 2.0 | ES: 6.0 | DF: 3.0 | CE: 7.0 | TO: 2.0 | GO: 3.0 | AP: 3.0 | PA: 4.0 | AM: 2.0 | RO

In [ ]:
#três equipes

CTFPFunction('https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/congress-2010-v1.g',
             'https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/state-as-skill/2010.txt',
             'https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/party-as-skill/2010.txt',
             3,
             5,
             numUsersSep="\t")

Número usuários: 545
Matriz de relacionamentos [[0 1 1 ... 0 1 1]
 [1 0 1 ... 0 1 0]
 [1 1 0 ... 0 1 1]
 ...
 [0 0 0 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 0 1 ... 0 0 0]]


Número de skills: 27
Lista de skills: {0: 'MG', 1: 'SP', 2: 'RJ', 3: 'BA', 4: 'PE', 5: 'RR', 6: 'MS', 7: 'PR', 8: 'SC', 9: 'RS', 10: 'MT', 11: 'ES', 12: 'DF', 13: 'CE', 14: 'TO', 15: 'GO', 16: 'AP', 17: 'PA', 18: 'AM', 19: 'RO', 20: 'AC', 21: 'AL', 22: 'MA', 23: 'RN', 24: 'PB', 25: 'SE', 26: 'PI'}


Demanda de cada time: [[11 14 10  9  5  2  2  6  4  6  2  2  2  5  2  3  2  4  2  2  2  2  4  2
   2  2  2]
 [11 14 10  9  5  2  2  6  4  6  2  2  2  5  2  3  2  4  2  2  2  2  4  2
   2  2  2]
 [11 14 10  9  5  2  2  6  4  6  2  2  2  5  2  3  2  4  2  2  2  2  4  2
   2  2  2]]




## 2011

In [ ]:
# duas equipes 

CTFPFunction('https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/congress-2011-v1.g',
             'https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/state-as-skill/2011.txt',
             'https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/party-as-skill/2011.txt',
             2,
             5,
             numUsersSep="\t")

Número usuários: 553
Matriz de relacionamentos [[0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 [1 1 1 ... 1 0 0]]


Número de skills: 27
Lista de skills: {0: 'MG', 1: 'BA', 2: 'PE', 3: 'DF', 4: 'CE', 5: 'PI', 6: 'AM', 7: 'RJ', 8: 'SP', 9: 'PR', 10: 'SC', 11: 'RS', 12: 'ES', 13: 'GO', 14: 'MT', 15: 'AP', 16: 'PA', 17: 'AC', 18: 'TO', 19: 'RR', 20: 'MA', 21: 'RN', 22: 'PB', 23: 'AL', 24: 'SE', 25: 'MS', 26: 'RO'}


Demanda de cada time: [[11  8  5  2  4  2  2 10 15  6  4  6  2  4  2  2  4  2  2  2  4  2  2  2
   2  2  2]
 [11  8  5  2  4  2  2 10 15  6  4  6  2  4  2  2  4  2  2  2  4  2  2  2
   2  2  2]]




Total de indivíduos alocados: 535.0

Resultados da equipe 0
Número de indivíduos na equipe: 133.0
Número de indíviduos em cada skill: MG: 11.0 | BA: 9.0 | PE: 5.0 | DF: 2.0 | CE: 15.0 | PI: 2.0 | AM: 2.0 | RJ: 10.0 | SP: 15.0 | PR: 8.0 | SC: 4.0 | RS: 6.0 | ES: 2.0 | GO: 4.0 | MT: 2.0 | AP: 2.0 | PA: 9.0 | AC: 2.0 | TO: 2.0 | RR:

In [ ]:
#três equipes

CTFPFunction('https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/congress-2011-v1.g',
             'https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/state-as-skill/2011.txt',
             'https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/party-as-skill/2011.txt',
             3,
             3.8,
             numUsersSep="\t")

Número usuários: 553
Matriz de relacionamentos [[0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 [1 1 1 ... 1 0 0]]


Número de skills: 27
Lista de skills: {0: 'MG', 1: 'BA', 2: 'PE', 3: 'DF', 4: 'CE', 5: 'PI', 6: 'AM', 7: 'RJ', 8: 'SP', 9: 'PR', 10: 'SC', 11: 'RS', 12: 'ES', 13: 'GO', 14: 'MT', 15: 'AP', 16: 'PA', 17: 'AC', 18: 'TO', 19: 'RR', 20: 'MA', 21: 'RN', 22: 'PB', 23: 'AL', 24: 'SE', 25: 'MS', 26: 'RO'}


Demanda de cada time: [[14 11  7  3  6  3  2 14 19  8  6  8  3  6  3  2  5  2  2  2  6  2  3  2
   2  2  2]
 [14 11  7  3  6  3  2 14 19  8  6  8  3  6  3  2  5  2  2  2  6  2  3  2
   2  2  2]
 [14 11  7  3  6  3  2 14 19  8  6  8  3  6  3  2  5  2  2  2  6  2  3  2
   2  2  2]]




## 2012

In [ ]:
#duas equipes

CTFPFunction('https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/congress-2012-v1.g',
             'https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/state-as-skill/2012.txt',
             2,
             6,
             numUsersSep="\t")

## 2013

In [ ]:
CTFPFunction('https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/congress-2013-v1.g',
             'https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/state-as-skill/2013.txt',
             2,
             5,
             numUsersSep="\t")